In [8]:
import oauth2 as oauth
import urllib.request as urllib

# See assignment1.html instructions or README for how to get these credentials

api_key = "p2z0G1qAtyWj7vuRXYUlVBJZx"
api_secret = "IPkPFmI6WJpGKzpy2GoQKnqwej42vS5kxS0eFsd9gzlM9ByU7m"
access_token_key = "2738128121-aGAiXd7iR5ER0TyY35iUEzFMPxRmbQlPBhkTLwT"
access_token_secret = "jwlYHAUJyJr0h1gsg3Qa1LpdqlVvBra8k9h5eThRw3DMs"

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"


http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)



In [10]:
'''
Construct, sign, and open a twitter request
using the hard-coded credentials above.
'''
def twitterreq(url, method, parameters):
  req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

  req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

  headers = req.to_header()

  if http_method == "POST":
    encoded_post_data = req.to_postdata()
  else:
    encoded_post_data = None
    url = req.to_url()

  opener = urllib.OpenerDirector()
  opener.add_handler(http_handler)
  opener.add_handler(https_handler)

  response = opener.open(url, encoded_post_data)

  return response

def fetchsamples():
  url = "https://stream.twitter.com/1/statuses/sample.json"
  parameters = []
  response = twitterreq(url, "GET", parameters)
  for line in response:
    print(line.strip())

if __name__ == '__main__':
  fetchsamples()


In [14]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json

# create a dictionary to store your twitter credentials

twitter_cred = dict()

twitter_cred['API_KEY'] = 'p2z0G1qAtyWj7vuRXYUlVBJZx'
twitter_cred['API_SECRET'] = 'IPkPFmI6WJpGKzpy2GoQKnqwej42vS5kxS0eFsd9gzlM9ByU7m'
twitter_cred['ACCESS_TOKEN_KEY'] = '2738128121-aGAiXd7iR5ER0TyY35iUEzFMPxRmbQlPBhkTLwT'
twitter_cred['ACCESS_TOKEN_SECRET'] = 'jwlYHAUJyJr0h1gsg3Qa1LpdqlVvBra8k9h5eThRw3DMs'

# Save the information to a json so that it can be reused in code without exposing
# the secret info to public

with open('twitter_credentials.json', 'w') as secret_info:
    json.dump(twitter_cred, secret_info, indent=4, sort_keys=True)

In [21]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import tweepy
import csv
import json

# load Twitter API credentials

with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    api_key = info['API_KEY']
    api_secret = info['API_SECRET']
    access_token_key = info['ACCESS_TOKEN_KEY']
    access_token_secret = info['ACCESS_TOKEN_SECRET']

def get_all_tweets(screen_name):

    # Twitter allows access to only 3240 tweets via this method

    # Authorization and initialization

    auth = tweepy.OAuthHandler(api_key, api_secret)
    auth.set_access_token(access_token_key, access_token_secret)
    api = tweepy.API(auth)

    # initialization of a list to hold all Tweets

    all_the_tweets = []

    # We will get the tweets with multiple requests of 200 tweets each

    new_tweets = api.user_timeline(screen_name=screen_name, count=10)

    # saving the most recent tweets

    all_the_tweets.extend(new_tweets)

    # save id of 1 less than the oldest tweet

    oldest_tweet = all_the_tweets[-1].id - 1

    # grabbing tweets till none are left

    while len(new_tweets):
        # The max_id param will be used subsequently to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name,
        count=10, max_id=oldest_tweet)

        # save most recent tweets

        all_the_tweets.extend(new_tweets)

        # id is updated to oldest tweet - 1 to keep track

        oldest_tweet = all_the_tweets[-1].id - 1
        print ('...%s tweets have been downloaded so far' % len(all_the_tweets))

        # transforming the tweets into a 2D array that will be used to populate the csv

        outtweets = [[tweet.id_str, tweet.created_at,
        tweet.text.encode('utf-8')] for tweet in all_the_tweets]

        # writing to the csv file

        with open(screen_name + '_tweets.csv', 'w', encoding='utf8') as f:
            writer = csv.writer(f)
            writer.writerow(['id', 'created_at', 'text'])
            writer.writerows(outtweets)

if __name__ == '__main__':

# Enter the twitter handle of the person concerned

    get_all_tweets(input("Enter the twitter handle of the person whose tweets you want to download:- "))

Enter the twitter handle of the person whose tweets you want to download:- BarackObama
...20 tweets have been downloaded so far
...30 tweets have been downloaded so far
...40 tweets have been downloaded so far
...50 tweets have been downloaded so far
...60 tweets have been downloaded so far
...70 tweets have been downloaded so far
...80 tweets have been downloaded so far
...90 tweets have been downloaded so far
...100 tweets have been downloaded so far
...110 tweets have been downloaded so far
...120 tweets have been downloaded so far
...130 tweets have been downloaded so far
...140 tweets have been downloaded so far
...150 tweets have been downloaded so far
...160 tweets have been downloaded so far
...170 tweets have been downloaded so far
...180 tweets have been downloaded so far
...190 tweets have been downloaded so far
...200 tweets have been downloaded so far
...210 tweets have been downloaded so far
...220 tweets have been downloaded so far
...230 tweets have been downloaded so f

KeyboardInterrupt: 

In [ ]:
BarackObama